In [3]:
# % reset
import numpy as np
import matplotlib.pyplot as plt

import keras

from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout


## Get the data and pre-process it

In [4]:
(X_train, y_train), (X_test, y_test)=mnist.load_data()
X_train.shape , y_train.shape , X_test.shape , y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
def plot_input_img(i):
    plt.imshow(X_train[i] , cmap = 'binary')
    plt.title(y_train[i])
    plt.show()

In [6]:
# Pre Process the images

# Normalizing the image to [0 , 1] range
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255

# Reshape / expand the dimetions of image to (28,28,1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

#  convert classes to one hot vectors

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [7]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (28,28,1),activation='relu',padding='same'),)
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dropout(0.25))

model.add(Dense(10,activation="softmax"))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dropout (Dropout)           (None, 2304)              0

In [9]:
model.compile(optimizer='adam',loss = keras.losses.categorical_crossentropy , metrics=['accuracy'])

In [10]:
# Callbacks

from keras.callbacks import EarlyStopping , ModelCheckpoint

# Earlystopping

es= EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience=4,verbose=1)

# Model Check Point

mc = ModelCheckpoint("temp/bestmodel.h5",monitor="val_accuracy",verbose=1, save_best_only= True)

cb = [es , mc]

# Model Training

In [11]:
his = model.fit(X_train, y_train,epochs=10,validation_split=0.3, callbacks=cb ,batch_size=64)

Epoch 1/10
656/657 [============================>.] - ETA: 0s - loss: 0.2644 - accuracy: 0.9205
Epoch 1: val_accuracy improved from -inf to 0.97217, saving model to temp\bestmodel.h5
657/657 [==============================] - 21s 32ms/step - loss: 0.2643 - accuracy: 0.9205 - val_loss: 0.0917 - val_accuracy: 0.9722
Epoch 2/10
655/657 [============================>.] - ETA: 0s - loss: 0.0825 - accuracy: 0.9748
Epoch 2: val_accuracy improved from 0.97217 to 0.97789, saving model to temp\bestmodel.h5
657/657 [==============================] - 22s 33ms/step - loss: 0.0825 - accuracy: 0.9747 - val_loss: 0.0713 - val_accuracy: 0.9779
Epoch 3/10
655/657 [============================>.] - ETA: 0s - loss: 0.0630 - accuracy: 0.9805
Epoch 3: val_accuracy improved from 0.97789 to 0.98383, saving model to temp\bestmodel.h5
657/657 [==============================] - 21s 32ms/step - loss: 0.0631 - accuracy: 0.9805 - val_loss: 0.0536 - val_accuracy: 0.9838
Epoch 4/10
655/657 [==========================

In [15]:
model_S = keras.models.load_model("temp/bestmodel.h5")

In [16]:
score = model_S.evaluate(X_test,y_test)

print(f" the model accuracy is {score[1]}")

313/313 [==============================] - 2s 6ms/step - loss: 0.0304 - accuracy: 0.9899
 the model accuracy is 0.9898999929428101


In [14]:
# import numpy as np
# import cv2
# from tensorflow.keras.preprocessing import image
# lab_rand = class_names[np.random.randint(0,47)] #50種挑其中一類
# testdir = os.path.join(base_dirtest,lab_rand) #選出所在種類的資料路徑
# #選出其中一張照片的路徑
# testimg=os.path.join(testdir,os.listdir(testdir)[np.random.randint(0,len(testdir)-1)]) 
# x = image.load_img(testimg,target_size=(img_size,img_size))
# #模型輸出種類及預測機率 top-5
# x = np.expand_dims(x, axis = 0) 
# pred = model.predict(x)[0]
# top_inds = pred.argsort()[::-1][:5]
# for i in top_inds:
#   print('    {:.3f}  {}'.format(pred[i], class_names[i]))
# plt.subplot(1, 1, 1)
# image = cv2.imread(testimg)
# image = image[:,:,::-1]
# plt.imshow(image)